In [1]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import folium

In [2]:
import osmnx as ox
import networkx as nx
import folium
import pandas as pd

# Coordenadas do Campus Samambaia da UFG
campus_coords = (-16.603422, -49.265381)  # Latitude e longitude aproximadas

# Definir o raio em metros
radius = 2000  # 2 km

# Coletar o grafo de ruas dentro do raio especificado
G = ox.graph_from_point(campus_coords, dist=radius, network_type='drive')

# Extrair nós e arestas
nodes, edges = ox.graph_to_gdfs(G)


In [3]:
# Centro do mapa baseado no ponto central definido
center = campus_coords

# Criar o mapa com folium usando tiles leves
m = folium.Map(location=center, zoom_start=14, tiles='cartodbpositron')

# Adicionar arestas ao mapa de forma eficiente
for _, row in edges.iterrows():
    points = [(lat, lon) for lat, lon in zip(row['geometry'].xy[1], row['geometry'].xy[0])]
    folium.PolyLine(points, color='blue', weight=1.0, opacity=0.6).add_to(m)

# Adicionar nós ao mapa de forma eficiente
for _, row in nodes.iterrows():
    folium.CircleMarker(location=(row['y'], row['x']), radius=1, color='red', fill=True).add_to(m)

# Mostrar o mapa
m.save('map_optimized_campus.html')


In [4]:
# Função para encontrar a rota mais curta
def find_approx_route(G, orig, dest):
    # Use um algoritmo de busca aproximada se necessário
    return nx.shortest_path(G, orig, dest, weight='length')

# Definir os nós de origem e destino
orig_node = list(G.nodes())[0]
dest_node = list(G.nodes())[-1]

# Encontrar a rota mais curta
shortest_route = find_approx_route(G, orig_node, dest_node)

# Extrair coordenadas da rota mais curta
route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_route]

# Adicionar a rota mais curta ao mapa
folium.PolyLine(route_coords, color='green', weight=2.5, opacity=0.8).add_to(m)

# Mostrar o mapa com a rota
m.save('map_with_approx_route_campus.html')


In [5]:
# Visualizar os primeiros nós
display(nodes.head())

,y,x,street_count,highway,geometry
osmid,,,,,
340270144,-16.618710,-49.283693,3,NaN,POINT (-49.28369 -16.61871)
340270148,-16.603469,-49.280561,3,NaN,POINT (-49.28056 -16.60347)
340270840,-16.621125,-49.283894,3,NaN,POINT (-49.28389 -16.62113)
340271710,-16.599199,-49.281975,3,NaN,POINT (-49.28198 -16.5992)
1046749480,-16.602250,-49.268520,3,NaN,POINT (-49.26852 -16.60225)


In [6]:
# Visualizar os primeiros atributos das arestas
display(edges.head())

osmid  \
u         v          key                           
340270144 3095000770 0                 304930948   
          4033851168 0                 465225861   
          3677117783 0                 923759035   
340270148 8540152564 0    [279248147, 279248148]   
          4824435322 0                 490268036   

                                                  name      highway  oneway  \
u         v          key                                                      
340270144 3095000770 0    Rua Maria Josefina Vaiandeli  residential   False   
          4033851168 0               Avenida Nerópolis      primary   False   
          3677117783 0               Avenida Nerópolis      primary   False   
340270148 8540152564 0           [Rua Finlâdia, Rua 2]     tertiary   False   
          4824435322 0                             NaN      primary    True   

                         reversed   length  \
u         v          key                     
340270144 3095000770 0      False   77.840   
          4033851168 0      False   60.101   
          3677117783 0       True  159.108   
340270148 8540152564 0      False  831.756   
          4824435322 0      False    8.524   

                                                                   geometry  \
u         v          key                                                      
340270144 3095000770 0    LINESTRING (-49.28369 -16.61871, -49.28298 -16...   
          4033851168 0    LINESTRING (-49.28369 -16.61871, -49.28372 -16...   
          3677117783 0    LINESTRING (-49.28369 -16.61871, -49.28367 -16...   
340270148 8540152564 0    LINESTRING (-49.28056 -16.60347, -49.28056 -16...   
          4824435322 0    LINESTRING (-49.28056 -16.60347, -49.28053 -16...   

                            junction  ref maxspeed lanes bridge  
u         v          key                                         
340270144 3095000770 0           NaN  NaN      NaN   NaN    NaN  
          4033851168 0           NaN  NaN      NaN   NaN    NaN  
          3677117783 0           NaN  NaN      NaN   NaN    NaN  
340270148 8540152564 0           NaN  NaN      NaN   NaN    NaN  
          4824435322 0    roundabout  NaN      NaN   NaN    NaN